<a href="https://colab.research.google.com/github/hkdan502/NLP/blob/main/pair_cls_train_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지 설치하기

TPU 관련 패키지를 설치합니다. TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요. GPU를 쓴다면 아래 라인을 실행할 필요가 없습니다.

In [16]:
# TPU 사용시 아래 라인 첫 문자(#)를 지우고 수행하세요.
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

TPU 이외에 의존성 있는 패키지를 설치합니다.

In [ ]:
!pip install ratsnlp

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [18]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive
Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [19]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_task_name="pair-classification", #for NLI
    downstream_corpus_name="klue-nli", #dataset for NLI
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=5,
    #tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [20]:
args

ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=4, cpu_workers=2, fp16=False, tpu_cores=0)

ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=4, cpu_workers=2, fp16=False, tpu_cores=0)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [21]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7
set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [22]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=4, cpu_workers=2, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size

# 말뭉치 다운로드
실습에 사용할 말뭉치를 다운로드합니다.

In [23]:
nlpbook.download_downstream_dataset(args) #KLUE-NLI download

Downloading: 100%|██████████| 12.3M/12.3M [00:00<00:00, 16.2MB/s]
Downloading: 100%|██████████| 1.47M/1.47M [00:00<00:00, 28.9MB/s]
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [24]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

# 학습데이터 구축
학습데이터를 만듭니다.

- KlueNLICorpus는 JSON형식의 KLUE-NLI 데이터를 문장(전제, 가설)과 레이블(참, 거짓)으로 읽는다. 

- ClassificationDataset은 KlueNLICorpus가 넘겨준 문장(전제, 가설)과 레이브르 각각을 모델이 학습가능한 형태("*ClassifictionFeatures")로 가공해준다. 즉, 전제와 가설 2 문장을 각각 토큰화하고 이를 인덱스로 변환하면서 레이블 역시 정수(entailment:0, contradiction:1, neutral:2)로 바꿔주는 역할을 수행한다. 

- *ClassificationFeatures : 데이터 인스턴스 x를 input_ids, attention_mask, token_type_ids, label등 4가지로 변환한 자료 

In [25]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
#학습용 데이터 로더
#배치 크기(args의 batch_size)만큼의 인스턴스들을 비복원(replacement=False) 랜덤추출(RandomSampler)해서 배치 형태로 가공(nlpbook.data_collator)에 모델에 공급
train_dataloader = DataLoader( 
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 12.926 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 100분간 잘껄 그래도 소닉붐땜에 2점준다 + 100분간 잤다.
INFO:ratsnlp:tokens: [CLS] 100 ##분간 잘 ##껄 그래도 소 ##닉 ##붐 ##땜에 2 ##점 ##준다 [SEP] 100 ##분간 잤 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [35]:
train_dataset[0] #label=1 : contradiction

'''
- input_ids는 '[CLS] 전제 [SEP] 가설 [SEP]'를 토큰화한 뒤 인덱싱한 결과
- attention_mask는 해당 위치의 토큰이 패딩 토큰(0)인지 아닌지(1: 실제데이터)
- token_type_ids는 segment정보로, 'CLS premise SEP'에 해당하는 첫 세그먼트는 0, 'hypothesis SEP'에 해당하는 두번째 세그먼트는 1, 나머지 패딩에 속하는 세번째 세그먼트는 다시 0을 준다.
전제, 가설의 토큰수가 각각 12,5개 이므로 첫번째 세그먼트(0)의 길이는 cls, sep을 합쳐 14,
두번째 세그먼트(1)의 길이는 sep을 포함해 6이 된다. 마지막 세그먼트(0)의 길이는 64(max_length)-14-6=44가 된다.

'''

"\n- input_ids는 '[CLS] 전제 [SEP] 가설 [SEP]'를 토큰화한 뒤 인덱싱한 결과\n- attention_mask는 해당 위치의 토큰이 패딩 토큰(0)인지 아닌지(1: 실제데이터)\n- token_type_ids는 segment정보로, 'CLS premise SEP'에 해당하는 첫 세그먼트는 0, 'hypothesis SEP'에 해당하는 두번째 세그먼트는 1, 나머지 패딩에 속하는 세번째 세그먼트는 다시 0을 준다.\n전제, 가설의 토큰수가 각각 12,5개 이므로 첫번째 세그먼트(0)의 길이는 cls, sep을 합쳐 14,\n두번째 세그먼트(1)의 길이는 sep을 포함해 6이 된다. 마지막 세그먼트(0)의 길이는 64(max_length)-14-6=44가 된다.\n\n\n"

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [26]:
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)

#평가용 데이터 로더
#배치 크기(args의 batch_size)만큼의 인스턴스들을 순서대로 추출(Sequential Sampler)해서 배치 형태로 가공(nlpbook.data_collator)에 모델에 공급

val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from dataset file at /content/Korpora/klue-nli
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 1.604 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 10명이 함께 사용하기 불편함없이 만족했다. + 10명이 함께 사용하기 불편함이 많았다.
INFO:ratsnlp:tokens: [CLS] 10명 ##이 함께 사용 ##하기 불편 ##함 ##없이 만족 ##했다 . [SEP] 10명 ##이 함께 사용 ##하기 불편 ##함이 많았 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 21000, 4017, 9158, 9021, 8268, 10588, 4421, 8281, 14184, 8258, 17, 3, 21000, 4017, 9158, 9021, 8268, 10588, 11467, 14338, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 쌍 분류를 수행할 모델을 초기화합니다.

In [27]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [28]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

# 학습 준비
Task와 Trainer를 준비합니다.

In [29]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [30]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-paircls`)에 저장됩니다.

In [31]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:608: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-paircls exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:724: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
